# Analysis of Grades
#### for the University of Washington - Madison, WI
#### with Kaspar & Hussein Consulting

Dataset from [kaggle.com](https://www.kaggle.com/Madgrades/uw-madison-courses)

### To create new database from pgloader:
run these commands in the terminal:

    1.  brew install pgloader         # possibly already installed
    2.  createdb uw_mad_grades        # createdb {databasename}  
    3.  pgloader database.sqlite3 postgresql:///uw_mad_grades       # pgloader {sqllite3 filename} postgresql:///{database name}
    
We will be using a combination of PostgreSQL and also pandas.read_csv(...) since the data is available in both forms.
When we only need 1 table at a time, pandas DataFrame will be more convenient. Let's walk through the data folder and read
the files, saving them each to their own variable.

In [1]:
import pandas as pd
import numpy as np
import os
import psycopg2
import csv
from scipy import stats

Here we will find the file names where the raw data lives (inside /data folder).

In [2]:
file_names = []
for (dirpath, dirnames, filenames) in os.walk('data'):
    file_names.extend(filenames)
    break
    
file_names = list(filter(lambda x:'csv' in x, filenames))
len(file_names)

10

Now we build and save some dataframes to use later

In [3]:
tables = {}
for table in file_names:
    with open('data/' + table) as f:
        tables[table[:-4]] = pd.read_csv(f)
        
subjects = tables['subjects']
subject_memberships = tables['subject_memberships']
rooms = tables['rooms']
schedules = tables['schedules']
sections = tables['sections']
courses = tables['courses']
course_offerings = tables['course_offerings']
instructors = tables['instructors']
teachings = tables['teachings']
grade_distributions = tables['grade_distributions']
        
print(f'Converted into {len(tables)} dataframes stored in the dictionary called "tables" as well as their individual filenames')

Converted into 10 dataframes stored in the dictionary called "tables" as well as their individual filenames


Now we begin pulling data from Postgres, first we create a "helper-table" to cast a few important columns as numbers instead of strings.

In [4]:
conn = psycopg2.connect("dbname=uw_mad_grades")
cur = conn.cursor()
query = '''
DROP TABLE IF EXISTS grade_dist_numeric
'''
cur.execute(query)
query = '''
CREATE TABLE grade_dist_numeric AS (
  SELECT
    course_offering_uuid
  , section_number
  , CAST(grade_distributions.a_count AS integer)
  , CAST(grade_distributions.ab_count AS integer)
  , CAST(grade_distributions.b_count AS integer)
  , CAST(grade_distributions.bc_count AS integer)
  , CAST(grade_distributions.c_count AS integer)
  , CAST(grade_distributions.d_count AS integer)
  , CAST(grade_distributions.f_count AS integer)
  FROM grade_distributions
  )
'''
cur.execute(query)
conn.commit()

This is the big query that will get us all the data we need for the rest of the project. 
Protip: Formatting a query this large is much easier in a code editor such as Vim, since it will color code variable names and do other helpful things, whereas Jupyter will treat everything inside the string as raw text, which is harder to format nicely.

In [5]:
conn = psycopg2.connect("dbname=uw_mad_grades")
cur = conn.cursor()

query = '''
SELECT
  sections.course_offering_uuid
, course_offerings.course_uuid
, course_offerings.name
, COUNT(sections.number) AS section_count
, instructor_id
, SUM(a_count) AS a_count
, SUM(ab_count) AS ab_count
, SUM(b_count) AS b_count
, SUM(bc_count) AS bc_count
, SUM(c_count) AS c_count
, SUM(d_count) AS d_count
, SUM(f_count) AS f_count
FROM
   teachings
INNER JOIN
   sections
   ON teachings.section_uuid=sections.uuid
INNER JOIN
   grade_dist_numeric
   ON (
     (sections.course_offering_uuid = grade_dist_numeric.course_offering_uuid)
      AND (sections.number = grade_dist_numeric.section_number))
INNER JOIN
   course_offerings
   ON sections.course_offering_uuid=course_offerings.uuid
WHERE
   (a_count + ab_count + b_count + bc_count + c_count + d_count + f_count) > 0
GROUP BY
  sections.course_offering_uuid
, course_offerings.course_uuid
, course_offerings.name
, instructor_id
HAVING
  count(instructor_id) > 1;
'''

cur.execute(query)
all_grades_count_df = pd.DataFrame(cur.fetchall())
all_grades_count_df.columns = [i[0] for i in cur.description]

Each row of `all_grades_count_df` is an individual class taught be one particular professor, along with the grade counts given for each letter grade. Let's take a look:

In [6]:
all_grades_count_df.head(2)

,course_offering_uuid,course_uuid,name,section_count,instructor_id,a_count,ab_count,b_count,bc_count,c_count,d_count,f_count
0,000d1ef4-b78a-304e-8ff9-382dfc96835a,10362c9c-bbf8-39e3-8506-c2783ee31625,Diversity in Special Education,3,3802457,96,15,0,0,0,0,0
1,00193853-8f6d-3e14-994b-77a557b35d74,a4dcc153-1a04-39d8-9b28-b56fb4fa5266,Cult Anthro&Human Diversity,2,3409966,368,272,109,42,21,8,6


Let's build some new columns: `class_size` is pretty obvious, `counts` is less so. This is a column detailing how many times that class has been taught by *any* professor. If there's an "unpopular" class that was only that was only offered 3 times (regardless of whether it was taught by 1 prof three diff times or 3 profs one time each) this column would show `3`.
Finally, `pct_A` and `pct_F` are calculated per class, but later we will examine an individual professor's likelihood of giving A's and F's.

In [7]:
col = ['a_count', 'ab_count', 'b_count', 'bc_count', 'c_count', 'd_count', 'f_count']
all_grades_count_df['class_size'] = all_grades_count_df[col].sum(axis=1)
all_grades_count_df['counts'] = all_grades_count_df.groupby(['course_uuid'])['course_uuid'].transform('count')
all_grades_count_df["pct_A"] = all_grades_count_df['a_count'] / all_grades_count_df['class_size']
all_grades_count_df['pct_F'] = all_grades_count_df['f_count'] / all_grades_count_df['class_size']

Now we isolate the single course which has been offered the greatest number of times.

In [8]:
#Which class was taught the most times?
#all_grades_count_df = all_grades_count_df.loc[all_grades_count_df['counts']>1]
all_grades_count_df.sort_values('counts', ascending=False).head(1)

,course_offering_uuid,course_uuid,name,section_count,instructor_id,a_count,ab_count,b_count,bc_count,c_count,d_count,f_count,class_size,counts,pct_A,pct_F
7587,a1beab6b-05ea-3725-9456-80bed7367a73,7a552326-da08-354c-aafc-f56edcf83f44,Intro to Speech Composition,2,4286528,6,10,4,0,0,0,1,21,489,0.285714,0.047619


Cool, looks like "Intro to Speech Composition" was taught a total of 489 times. Let's suppose we are advising the Head of the Communication Arts Dept. at UW-Madison, who would like to know whether any professors in the department are "outliers" in terms of grades given. ie. Does anyone give A's "too freely" or alternatively do they give F's too readily? We will find list of professors from [the official website](https://commarts.wisc.edu/people?rid=All&field_person_affiliation_tid=11) and compare their percentage of a chosen grade (A or F) with the overall percentage of grades given for anyone who has taught the class.

In [9]:
prof_matters = all_grades_count_df.loc[all_grades_count_df['course_uuid'] == '7a552326-da08-354c-aafc-f56edcf83f44'].drop(['course_offering_uuid', 'course_uuid', 'section_count', 'name', 'counts'], axis=1)
prof_matters.head(0)     # same as prof_matters.columns   but easier to read

,instructor_id,a_count,ab_count,b_count,bc_count,c_count,d_count,f_count,class_size,pct_A,pct_F


In [10]:
current_profs = '''Paul Ahn, Joanne Cantor, Tammy Chang, Anthony Chen, Larisa Doroshenko, Mary Anne Fitzpatrick, Xinle Jia, Sangwon Lee, 
                    Yang Liu, Marie-Louise Mares, AnneMarie McClain, David Mortensen, Julian Mueller-Herbst, Esther Paik, Zhongdang Pan, 
                    Alanna Peebles, Irene Sarmiento, Maura Snyder, Catalina L. Toma, Fangjing Tu, Lyn Van Swol, Michael Xenos'''

current_profs = current_profs.split(',')
current_profs = [name.strip().upper() for name in current_profs]

current_profs_df = instructors.loc[instructors['name'].isin(current_profs)].copy()
current_profs_df

,id,name
258,5519194,IRENE SARMIENTO
266,5168073,ALANNA PEEBLES
332,5701690,JULIAN MUELLER-HERBST
1051,4145440,YANG LIU
2797,5334549,YANG LIU
8846,4715210,YANG LIU
8890,4909736,LARISA DOROSHENKO
8893,5186059,SANGWON LEE
8894,1591621,MICHAEL XENOS
8898,5181640,FANGJING TU


Now let's build a function to parse through *all* professors over the years that have ever taught "Intro to Speech Composition" so that we can compare the current instructors vs historical averages.

In [11]:
def prof_a_size(df=prof_matters, grade='a'):
    '''Input a dataframe with grade counts and the professors who gave those grades, along with the 
       exact grade letter of interest. Output two dictionaries, each with professor as the keys, 
       ( {prof : df} , {prof : [%<grade> of THIS prof, %<grade> of OTHER profs]})'''
    profs = df['instructor_id'].values
    profs_dict = {}
    profs_pct = {}
    grade_search = grade.lower() + '_count'
    for prof in profs:
        profs_dict[prof] = df.loc[df['instructor_id'] == prof, [grade_search, 'class_size']]   
        other_profs = df.loc[df['instructor_id'] != prof, [grade_search, 'class_size']]
        pct_other = round(other_profs.sum()[0] / other_profs.sum()[1], 4)
        
        pct_a = round(profs_dict[prof].sum()[0] / profs_dict[prof].sum()[1], 4)
        profs_pct[prof] = pct_a, pct_other
        
    return profs_dict, profs_pct

profs_dict, profs_pct = prof_a_size(prof_matters, grade='a')

What do these outputs look like? Here are the first 3 examples from each dictionary:

In [12]:
random_3 = np.random.choice(list(profs_dict.keys()), 3)
for prof in random_3:
    print(f'     Professor ID: {prof}')
    display(profs_dict[prof], profs_pct[prof])
    print(' ')
    print(' ')

     Professor ID: 3009080


,a_count,class_size
2593,2,23
3995,0,21


(0.0455, 0.2358)

 
 
     Professor ID: 1441959


,a_count,class_size
599,4,25
2589,6,25
3685,12,23
4762,5,25
7575,4,26


(0.25, 0.235)

 
 
     Professor ID: 5183812


,a_count,class_size
919,2,22
2689,3,23
5788,2,23


(0.1029, 0.2358)

In [24]:
def get_name_from_id(instructor_id):
    return instructors['name'].loc[instructors['id'] == int(instructor_id)].reset_index(drop=True)[0].title()

In [27]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

def prof_hypothesis_test(instructor_id, alpha=.05, grade='a', verbose=False, profs_dict=profs_dict, profs_pct=profs_pct):
    """input an instructor_id as either a number or a string, calculate the confidence interval for the ratio of a chosen 
       grade letter, and compute whether this professor gives this grade fairly or not fairly (at a specified alpha level)
       when compared to the rest of professors who teach the same class."""
    test_id = list(profs_dict.keys())[0]
    test_letter = profs_dict[test_id].columns[0][0]
    if test_letter != grade.lower():
        profs_dict, profs_pct = prof_a_size(prof_matters, grade=grade)
    if str(instructor_id) not in profs_dict.keys():
        return None, None
    
    name = get_name_from_id(instructor_id)
    
    instructor_id = str(instructor_id)
    
    extreme_prof = 'fair'
    instructor_As = profs_dict[instructor_id].sum()[0]
    instructor_students = profs_dict[instructor_id].sum()[1]
    other_instructor_pct = profs_pct[instructor_id][1]
    output = proportions_ztest(instructor_As, instructor_students, other_instructor_pct)
    
    if output[1] < alpha:
        extreme_prof = "not fair"
        
    if verbose:
        lower, upper = proportion_confint(instructor_As, instructor_students, alpha)
        print(f"{name.title()} is expected to give a ratio of {grade.upper()}'s bewteen {round(100*lower, 4)} and {round(100*upper, 4)}%.")
        print(f"{name.title()} was {extreme_prof}, and gave {round(100*instructor_As/instructor_students, 2)}% {grade.upper()}'s, vs other instructors: {round(100*other_instructor_pct, 2)}%")
        print(f"{instructor_As} {grade.upper()}'s out of {instructor_students} total students.")
        print('   ')
    return output

In [28]:
current_prof_ids = current_profs_df['id'].astype(str).to_list()
evaluate_current = prof_matters.loc[prof_matters['instructor_id'].isin(current_prof_ids)].copy()

In [29]:
evaluate_current['Z_score'] = evaluate_current['instructor_id'].apply(lambda x:prof_hypothesis_test(x)[0])
evaluate_current['p_value'] = evaluate_current['instructor_id'].apply(lambda x:prof_hypothesis_test(x)[1])
evaluate_current

,instructor_id,a_count,ab_count,b_count,bc_count,c_count,d_count,f_count,class_size,pct_A,pct_F,Z_score,p_value
1690,5701690,12,9,2,0,0,0,0,23,0.521739,0.0,2.756749,0.005838
5786,5168073,10,11,3,1,0,0,0,25,0.400000,0.0,1.686065,0.091783
5795,5519194,7,13,3,0,1,1,0,25,0.280000,0.0,0.501115,0.616290


In [30]:
for prof in evaluate_current['instructor_id']:
    prof_hypothesis_test(prof, verbose=True)

Julian Mueller-Herbst is expected to give a ratio of A's bewteen 31.7592 and 72.5886%.
Julian Mueller-Herbst was not fair, and gave 52.17% A's, vs other instructors: 23.46%
12 A's out of 23 total students.
   
Alanna Peebles is expected to give a ratio of A's bewteen 20.7964 and 59.2036%.
Alanna Peebles was fair, and gave 40.0% A's, vs other instructors: 23.48%
10 A's out of 25 total students.
   
Irene Sarmiento is expected to give a ratio of A's bewteen 10.3996 and 45.6004%.
Irene Sarmiento was fair, and gave 28.0% A's, vs other instructors: 23.5%
7 A's out of 25 total students.
   


In [ ]:
get_name_from_id

In [37]:
def extreme_profs(n=5):
    print("Easiest professors : (their % A's , other prof % A's)")
    print(' ')
    display([(get_name_from_id(key), val) for (key, val) in sorted(profs_pct.items(), key=lambda x: x[1][0], reverse=True)][:n])
    print(' ')
    print('----------------------------------')
    print(' ')
    print("Hardest professors : (their % A's , other prof % A's)")
    print(' ')
    display([(get_name_from_id(key), val) for (key, val) in sorted(profs_pct.items(), key=lambda x: x[1][0], reverse=True)][-n:])
extreme_profs(5)

Easiest professors : (their % A's , other prof % A's)
 


[('Joseph Strand', (0.8, 0.234)),
 ('Lindsay Hogan Garrison', (0.7551, 0.2331)),
 ('Leland Pan', (0.7345, 0.2305)),
 ('Douglas Adams', (0.6923, 0.2342)),
 ('Dominique Salas', (0.6667, 0.2343))]

 
----------------------------------
 
Hardest professors : (their % A's , other prof % A's)
 


[('Christopher Butler', (0.0408, 0.2359)),
 ('Joelle Tybon', (0.04, 0.2355)),
 ('Monica Styles', (0.04, 0.2355)),
 ('Victoria Lantz', (0.0385, 0.2355)),
 ('James Matenaer', (0.0312, 0.2362))]